# Single Rendevous Experiments with GTOC5

## Sorting out imports

In [1]:
import pykep as pk
import numpy as np
from tqdm import tqdm, trange

In [2]:
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 8

In [6]:
import src
import src.gtoc5
import src.gtoc5.gtoc5
from src.gtoc5.gtoc5 import mission_to_1st_asteroid, add_asteroid

# Basic steps for assembling GTOC5 trajectories

The two primary functions for assembling a GTOC5 trajectory are here `mission_to_1st_asteroid()` and `add_asteroid()`. The first initializes the mission's data structure with the details of the Earth launch leg, that takes the spacecraft towards the mission's first asteroid. Subsequently, via multiple calls to `add_asteroid()`, the mission is extended with additional exploration targets. Each call to `add_asteroid()` creates a rendezvous leg towards the specified asteroid, immediately followed by a flyby of that same asteroid, and so increases the mission's overall score by 1.

Here's an example of a mission that launches towards asteroid `1712`, and moves next to asteroid `4893`. The `True` value returned by `add_asteroid()` indicates that a feasible transfer leg was found, and asteroid `4893` was therefore added to the mission.

In [7]:
t = mission_to_1st_asteroid(1712)

[(1712, 3912.530999146394, 59127.205255048466, 59181.08381655966), (5029, 3901.4625422909457, 60656.59896026373, 60723.04042788253), (6259, 3900.756325308021, 60176.080828397084, 60243.3238638554), (5264, 3894.3554297926958, 57395.940540683376, 57470.448687286786), (5778, 3892.8693281905375, 58090.22128050112, 58166.416171671044), (5159, 3888.5440120099083, 60853.41483263781, 60934.519007401104), (6112, 3880.957416808325, 60970.97429870905, 61060.689322120365), (4374, 3872.8592343610107, 58107.18368566181, 58206.090176378544), (5447, 3868.707060537131, 60458.0836311495, 60561.702838445846), (5249, 3868.2577508310205, 58803.76483942432, 58907.89401250055), (2921, 3867.881958300564, 57889.924350001165, 57994.48004685408), (2713, 3867.0828852318996, 59922.6769960359, 60028.13963624344), (3480, 3864.960725177576, 57606.10710138145, 57713.978388709875), (4772, 3864.8952135297322, 59471.137508520456, 59579.08315129767), (4150, 3861.611707690246, 58748.26156381479, 58859.933968095946), (6865,

In [8]:
add_asteroid(t, 4893)

True

We can evaluate this trajectory with respect to its score (number of asteroids fully explored), final mass (in kg), and time of flight (here converted from days to years).

In [20]:
score(t), final_mass(t), tof(t) * DAY2YEAR

(2.0, 3484.7515275785117, 1.6150747703446979)

An aggregation of the mission's mass and time costs can be obtained with `resource_rating()`. It measures the extent to which the mass and time budgets available for the mission have been depleted by the trajectory. It produces a value of 1.0 at the start of the mission, and a value of 0.0 when the mission has exhausted its 3500 kg of available mass, or its maximum duration of 15 years.

In [21]:
resource_rating(t)

0.8721092603416899

As the score increments discretely by 1.0 with each added asteroid, and the resource rating evaluates mass and time available in a range of [0, 1], both can be combined to give a single-objective evaluation of the trajectory, that should be maximized:

In [22]:
score(t) + resource_rating(t)

2.87210926034169

Calling `seq()`, we can see either the full sequence of asteroids visited in each leg, or just the distinct asteroids visited in the mission. In this example, we see that the mission starts on Earth (id `0`), performs a rendezvous with asteroid `1712`, followed by a flyby of the same asteroid, and then repeats the pattern at asteroid `4893`.

In [23]:
print(seq(t))
print(seq(t, incl_flyby=False))

[0, 1712, 1712, 4893, 4893]
[0, 1712, 4893]


The trajectory data structure built by `mission_to_1st_asteroid()` and `add_asteroid()` is a list of tuples summarizing the evolution of the spacecraft's state. It provides the minimal sufficient information from which a more detailed view can be reproduced, if so desired. Each tuple contains:

1. asteroid ID
2. spacecraft mass
3. [epoch][epoch]
4. the leg's $\Delta T$
5. the leg's $\Delta V$

The mass and epoch values correspond to the state at the given asteroid, at the end of a rendezvous or self-fly-by leg, after deploying the corresponding payload. The $\Delta T$ and $\Delta V$ values refer to that leg that just ended.

[epoch]: https://en.wikipedia.org/wiki/Epoch_(astronomy)

In [24]:
t[-1]

(4893,
 3484.7515275785117,
 59717.111314916867,
 134.19998321371767,
 965.6854249492379)

Epochs are given here as [Modified Julian Dates (MJD)](https://en.wikipedia.org/wiki/Julian_day#Variants), and can be converted as:

In [25]:
pk.epoch(t[-1][2], 'mjd')

2022-May-18 02:40:17.608817

## Greedy search

In this section we perform a [Greedy search](https://en.wikipedia.org/wiki/Greedy_algorithm) for a GTOC5 trajectory. We'll start by going to asteroid `1712`. Then, and at every following step, we attempt to create legs towards all still available asteroids. Among equally-scored alternatives, we greedily pick the one with highest resource rating to adopt into the trajectory, and continue from there. Search stops when no feasible legs are found that can take us to another asteroid. This will happen either because no solutions were found that would allow for a leg to be created, or because adding a found solution would require the spacecraft to exceed the mission's mass or time budgets.

In [26]:
import os
from copy import copy

In [27]:
def greedy_step(traj):
    traj_asts = set(seq(traj, incl_flyby=False))
    progress_bar_args = dict(leave=False, file=os.sys.stdout, desc='attempting score %d' % (score(traj)+1))
    
    extended = []
    for a in trange(len(asteroids), **progress_bar_args):
        if a in traj_asts:
            continue
        tt = copy(traj)
        if add_asteroid(tt, next_ast=a, use_cache=False):
            extended.append(tt)
    
    return max(extended, key=resource_rating, default=[])

In [28]:
# measure time taken at one level to attempt legs towards all asteroids (that aren't already in the traj.)
%time _ = greedy_step(mission_to_1st_asteroid(1712))

Wall time: 11 s                                                                                                             


In [29]:
def greedy_search(first_ast):
    t = mission_to_1st_asteroid(first_ast)
    while True:
        tt = greedy_step(t)
        if tt == []:
            # no more asteroids could be added
            return t
        t = tt

In [30]:
%time T = greedy_search(first_ast=1712)

Wall time: 2min 30s                                                                                                         


In [31]:
score(T), resource_rating(T), final_mass(T), tof(T) * DAY2YEAR

(14.0, 0.041241978351621127, 584.8278465118636, 12.926528012051794)

In [32]:
print(seq(T, incl_flyby=False))

[0, 1712, 4893, 4028, 6939, 1059, 3295, 4006, 4981, 6437, 5344, 2090, 3993, 4186, 1130]


Greedy search gave us a trajectory that is able to visit 14 distinct asteroids. However, by the 14th, the spacecraft finds itself unable to find a viable target to fly to next, even though it has 84.8 kg of mass still available (the spacecraft itself weighs 500 kg, so the mission cannot go below that value), and 2 years remain in its 15 year mission.

## Phasing indicators

A big disadvantage of the approach followed above is the high computational cost of deciding which asteroid to go to next. It entails the optimization of up to 7075 legs, only to then pick a single one and discard all the other results.

An alternative is to use one of the indicators available in `gtoc5/phasing.py`. They can provide an indication of how likely a specific asteroid is to be an easily reachable target.

In [33]:
t = mission_to_1st_asteroid(1712)

We use here the (improved) orbital phasing indicator to rate destinations with respect to the estimated ΔV of hypothetical legs that would depart from `dep_ast`, at epoch `dep_t`, towards each possible asteroid, arriving there within `leg_dT` days. We don't know exactly how long the transfer time chosen by `add_asteroid()` would be, but we take `leg_dT=125` days as reference transfer time.

In [34]:
r = rate__orbital_2(dep_ast=t[-1][0], dep_t=t[-1][2], leg_dT=125)
r[seq(t)] = np.inf  # (exclude bodies already visited)

Below are the 5 asteroids the indicator estimates would be most easily reachable. As we've seen above in the results from the greedy search, asteroid `4893`, here the 2nd best rated alternative, would indeed be the target reachable with lowest ΔV.

In [35]:
r.argsort()[:5]

array([1679, 4893, 1528, 5331, 1663], dtype=int64)

The indicator is however not infallible. If we attempt to go from asteroid `1712` towards each of these asteroids, we find that none of them are actually reachable, except for `4893`! Still, the indicator allows us to narrow our focus considerably.

In [36]:
[add_asteroid(copy(t), a) for a in r.argsort()[:5]]

[False, True, False, False, False]

Armed with the indicator, we can reimplement the greedy search, so it will only optimize legs towards a number of top rated alternatives, and then proceed with the best out of those.

In [37]:
def narrowed_greedy_step(traj, top=10):
    traj_asts = set(seq(traj, incl_flyby=False))
    
    extended = []
    ratings = rate__orbital_2(dep_ast=traj[-1][0], dep_t=traj[-1][2], leg_dT=125)
    for a in ratings.argsort()[:top]:
        if a in traj_asts:
            continue
        tt = copy(traj)
        if add_asteroid(tt, next_ast=a, use_cache=False):
            extended.append(tt)
    
    return max(extended, key=resource_rating, default=[])

In [38]:
def narrowed_greedy_search(first_ast, **kwargs):
    t = mission_to_1st_asteroid(first_ast)
    while True:
        tt = narrowed_greedy_step(t, **kwargs)
        if tt == []:
            # no more asteroids could be added
            return t
        t = tt

In [39]:
# measure time taken at one level to attempt legs towards the best `top` asteroids
%time _ = narrowed_greedy_step(mission_to_1st_asteroid(1712), top=10)

Wall time: 105 ms


In [40]:
%time T = narrowed_greedy_search(first_ast=1712, top=10)

Wall time: 1.29 s


In [41]:
score(T), resource_rating(T), final_mass(T), tof(T) * DAY2YEAR

(14.0, 0.020976850009091538, 542.5005372055895, 13.845381094048889)

In [42]:
print(seq(T, incl_flyby=False))

[0, 1712, 4893, 4028, 6939, 1059, 505, 6060, 3907, 5051, 2413, 6819, 913, 6638, 6723]


We were able to find another score 14 trajectory, but this time it took us ~1 second, whereas before it was taking us 2 and a half minutes.

---

# Finding a GTOC5 trajectory of score 17 with Beam Search

In [17]:
gtoc_ph = init__path_handler(multiobj_evals=True)

In [18]:
# configuring Beam P-ACO to behave as a deterministic multi-objective Beam Search
_args = {
    'beam_width': 20,
    'branch_factor': 250,
    'alpha': 0.0,        # 0.0: no pheromones used
    'beta': 1.0,
    'prob_greedy': 1.0,  # 1.0: deterministic, greedy branching decisions
    }

In [19]:
bpaco = beam_paco_pareto(nr_nodes=len(asteroids), path_handler=gtoc_ph, random_state=None, **_args)

In [20]:
# start the search
# given we're running the algoritm in deterministic mode, we execute it for a single generation
%time best_pf = bpaco.solve(nr_generations=1)

Wall time: 2min 10s


In [21]:
# being this a `_pareto` class, .best returns a Pareto front
# pick the first solution from the Pareto front
best_eval, best = best_pf[0]

In [22]:
# Evaluation of the best found solution
# (score, mass consumed, time of flight)
best_eval

(17.0, 3496.8299418637093, 14.824982263414686)

In [25]:
# sequence of asteroids visited (0 is the Earth)
print(seq(best, incl_flyby=False))

[0, 1712, 4893, 2579, 6979, 5469, 6740, 2445, 6301, 5174, 5884, 4165, 4028, 6240, 3988, 1779, 6813, 3243]


In [24]:
# mission data structure, up to the full scoring of the first two asteroids
best[:5]

[(0, 4000.0, 59127.205255048466, 0.0, 0.0),
 (1712,
  3872.530999146394,
  59181.08381655966,
  53.87856151119195,
  650.4716282561097),
 (1712,
  3746.481928641157,
  59325.360311294986,
  144.27649473533114,
  965.6854249492379),
 (4893,
  3602.066962005158,
  59582.911331703152,
  257.55102040816325,
  831.5807789590044),
 (4893,
  3484.7515275785117,
  59717.111314916867,
  134.19998321371767,
  965.6854249492379)]

---

Generate the Table of Contents

In [43]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')
// https://github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>